In [1]:
import random
import sys
import time
from typing import List, Optional

from pyblend.algorithm.constructive import Constructive, LinModel, PostModel, PreModel
from pyblend.algorithm.heuristic import Heuristic, LAHC, SA
from pyblend.algorithm.neighborhood import (Shift, SimpleSwap, SmartShift,
                                            SmartSimpleSwap, SmartSwap, SmartSwitch,
                                            Swap, Switch)
from pyblend.config import Objective, Parameters
from pyblend.model.problem import Problem
from pyblend.model.solution import Solution
from pyblend.utils import handle_input_path, handle_output_path
from pyblend.main import feedback_approach, solve, construct, read_args


In [2]:
start_time = time.time()
parms: Parameters = {
    "constructive": "postmodel",
    "algorithm": "",
    "feedback": 0,
    "seed": 0,
    "maxiters": int(1e3),
    "lsize": int(1e3),
    "alpha": 0.9,
    "samax": int(1e3),
    "t0": 1.0,
}

In [6]:
random.seed(parms["seed"])

input_path = handle_input_path("instance_6.json")
output_path = handle_output_path("../out/json/out_6-2.json")

In [7]:
input_path

'/Users/erikingwersen/Desktop/github/PyBlend/tests/instance_6.json'

In [8]:
problem: Problem = Problem(input_path)

In [9]:
len(problem.stockpiles)

10

In [10]:
parms["constructive"]

'postmodel'

In [11]:
solution: Solution = Solution(problem)

In [12]:
problem.info[1:]

[1000, 1]

In [13]:
model: LinModel = LinModel(problem)

In [14]:
from mip import xsum

# Minimize the total scheduling time
total_time = xsum(
    model._x[i, k] * model._w_x[i, k] for i in range(model._p) for k in
    range(model._r)
)
total_time

In [18]:
total_time.x

In [15]:
total_time.expr

{<mip.entities.Var at 0x7fe668903a30>: 1,
 <mip.entities.Var at 0x7fe6689039d0>: 1,
 <mip.entities.Var at 0x7fe668903820>: 1,
 <mip.entities.Var at 0x7fe668903790>: 1,
 <mip.entities.Var at 0x7fe668903b20>: 1,
 <mip.entities.Var at 0x7fe668903850>: 1,
 <mip.entities.Var at 0x7fe668903be0>: 1,
 <mip.entities.Var at 0x7fe668903c10>: 1,
 <mip.entities.Var at 0x7fe668903c40>: 1,
 <mip.entities.Var at 0x7fe668903c70>: 1}

In [19]:
constructive: Constructive = construct(problem, solution, model, parms)

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 28 2021 

Starting solution of the Linear programming problem using Primal Simplex



In [20]:
constructive.solution.work_time(1)

(0.06, 91.04)

In [21]:
constructive._problem.outputs[0].weight / sum([
                eng.speed_reclaim for eng in constructive.problem.engines
            ])

76.92307692307692

In [20]:
_p: int = len(problem.stockpiles)
_t: int = len(problem.outputs[0].quality)
_r: int = len(problem.outputs)
_e: int = len(problem.inputs)
_p

problem.stockpiles

[id: 1
 position: 0
 yard: 1
 rails: [1]
 capacity: 100000
 weightIni: 60000
 qualityIni: [parameter: Fe
 value: 10.3
 , parameter: SiO2
 value: 2.3
 , parameter: Al2O3
 value: 4.5
 , parameter: P
 value: 0.05
 , parameter: +31.5
 value: 4
 , parameter: -6.3
 value: 40
 ],
 id: 2
 position: 1
 yard: 1
 rails: [1]
 capacity: 50000
 weightIni: 45000
 qualityIni: [parameter: Fe
 value: 76.3
 , parameter: SiO2
 value: 5.6
 , parameter: Al2O3
 value: 4.8
 , parameter: P
 value: 0.05
 , parameter: +31.5
 value: 8
 , parameter: -6.3
 value: 20
 ],
 id: 3
 position: 2
 yard: 2
 rails: [1]
 capacity: 100000
 weightIni: 100000
 qualityIni: [parameter: Fe
 value: 87.3
 , parameter: SiO2
 value: 3.7
 , parameter: Al2O3
 value: 5
 , parameter: P
 value: 0.05
 , parameter: +31.5
 value: 8.5
 , parameter: -6.3
 value: 15
 ],
 id: 4
 position: 3
 yard: 2
 rails: [1]
 capacity: 200000
 weightIni: 50000
 qualityIni: [parameter: Fe
 value: 56.3
 , parameter: SiO2
 value: 3.8
 , parameter: Al2O3
 value: 5

In [12]:
xsum(
    model._x[i, k] * model._w_x[i, k] for i in range(model._p) for k in
    range(model._r)
)

In [22]:
solver: Optional[Heuristic] = None
if parms["algorithm"] != "":
    solver = solve(problem, solution, constructive, parms)
    solution = solver.best_solution

if parms["feedback"] > 0:
    feedback_approach(solution, model, solver, constructive, parms)

solution.set_deliveries()
# solution.write(output_path, round(time.time()-start_time, 2))

In [24]:
total_time.x / 1_000

499.99999999999994

In [25]:
output_path

'../out/json/out_6-2.json'

In [26]:
solution.write(output_path, round(time.time()-start_time, 2))

In [28]:
total_time.x

499999.99999999994

In [30]:
[model._w_x[i, k] for i in range(model._p) for k in range(model._r)]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [32]:
[model._x[i, k].x for i in range(model._p) for k in range(model._r)]

[60000.00000000001,
 45000.0,
 78333.3333333327,
 46666.666666667275,
 59999.99999999999,
 75000.0,
 40000.0,
 45000.0,
 50000.0,
 0.0]

In [33]:
i_scheduling = xsum(
    model._w_y[h, i] * model._y[h, i]
    for h in range(model._e)
    for i in range(model._p)
)
i_scheduling.x

0.0

In [51]:
{
    f"id: {model._outputs[k].id}": [model._x[i, k].x for i in range(model._p)]
    for k in range(model._r)
}

{'id: 1': [60000.00000000001,
  45000.0,
  78333.3333333327,
  46666.666666667275,
  59999.99999999999,
  75000.0,
  40000.0,
  45000.0,
  50000.0,
  0.0]}

In [68]:
problem.engines


[id: 1
 speedStack: 0.0
 speedReclaim: 3900.0
 posIni: 8
 rail: 1
 yards: [1, 2, 3, 4, 5],
 id: 2
 speedStack: 0.0
 speedReclaim: 2600.0
 posIni: 7
 rail: 1
 yards: [1, 2, 3, 4, 5]]

In [69]:
model._e

1

In [62]:
solution._routes

[[(1, 'r'), (0, 'r'), (6, 'r'), (5, 'r'), (8, 'r')],
 [(2, 'r'), (3, 'r'), (4, 'r'), (7, 'r')]]

In [66]:
max([x['start_time'] + x['duration'] for x in solution._reclaims])

91.04

In [60]:
solution._problem.engines

[id: 1
 speedStack: 0.0
 speedReclaim: 3900.0
 posIni: 8
 rail: 1
 yards: [1, 2, 3, 4, 5],
 id: 2
 speedStack: 0.0
 speedReclaim: 2600.0
 posIni: 7
 rail: 1
 yards: [1, 2, 3, 4, 5]]

In [61]:

type(solution)

pyblend.model.solution.Solution

In [49]:
for engine in solution._problem.engines:
    routes.append(solution.set_route(start_time, engine))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [59]:
{
    f"id: {model._outputs[k].id}": [model._x[i, k].x for i in range(model._p)]
    for k in range(model._r)
}

{'id: 1': [60000.00000000001,
  45000.0,
  78333.3333333327,
  46666.666666667275,
  59999.99999999999,
  75000.0,
  40000.0,
  45000.0,
  50000.0,
  0.0]}

In [58]:
{
    f"id: {k.id}": [i.weight_ini for i in problem.stockpiles]
    for k in problem.outputs
}

{'id: 1': [60000,
  45000,
  100000,
  50000,
  60000,
  75000,
  40000,
  45000,
  50000,
  55000]}

In [53]:
solution._reclaims

[{'weight': 45000.0,
  'stockpile': 2,
  'engine': 1,
  'start_time': 0.06,
  'duration': 11.54,
  'output': 1},
 {'weight': 60000.0,
  'stockpile': 1,
  'engine': 1,
  'start_time': 12.81,
  'duration': 15.38,
  'output': 1},
 {'weight': 40000.0,
  'stockpile': 7,
  'engine': 1,
  'start_time': 29.09,
  'duration': 10.26,
  'output': 1},
 {'weight': 75000.0,
  'stockpile': 6,
  'engine': 1,
  'start_time': 40.15,
  'duration': 19.23,
  'output': 1},
 {'weight': 50000.0,
  'stockpile': 9,
  'engine': 1,
  'start_time': 59.88,
  'duration': 12.82,
  'output': 1},
 {'weight': 78333.3,
  'stockpile': 3,
  'engine': 2,
  'start_time': 0.06,
  'duration': 30.13,
  'output': 1},
 {'weight': 46666.7,
  'stockpile': 4,
  'engine': 2,
  'start_time': 31.4,
  'duration': 17.95,
  'output': 1},
 {'weight': 60000.0,
  'stockpile': 5,
  'engine': 2,
  'start_time': 50.15,
  'duration': 23.08,
  'output': 1},
 {'weight': 45000.0,
  'stockpile': 8,
  'engine': 2,
  'start_time': 73.73,
  'duration': 

In [41]:
problem._time_travel

[[0.06, 1.21, 0.16, 0.08, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
 [1.21, 0.06, 0.08, 0.16, 0.7, 0.8, 0.9, 1.0, 0.5, 0.6],
 [0.16, 0.08, 0.06, 1.21, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3],
 [0.08, 0.16, 1.21, 0.06, 0.9, 1.0, 0.5, 0.6, 0.7, 0.8],
 [0.5, 0.7, 0.8, 0.9, 0.06, 1.21, 0.16, 0.08, 0.5, 0.6],
 [0.6, 0.8, 0.7, 1.0, 1.21, 0.06, 0.08, 0.16, 0.7, 0.8],
 [0.7, 0.9, 0.6, 0.5, 0.16, 0.08, 0.06, 1.21, 0.16, 0.08],
 [0.8, 1.0, 0.5, 0.6, 0.08, 0.16, 1.21, 0.06, 0.8, 0.9],
 [0.9, 0.5, 0.4, 0.7, 0.5, 0.7, 0.16, 0.8, 0.06, 1.21],
 [1.0, 0.6, 0.3, 0.8, 0.6, 0.8, 0.08, 0.9, 1.21, 0.06]]

In [42]:
solution._problem.outputs

[id: 1
 destination: 1
 weight: 500000
 quality: [parameter: Fe
 minimum: 60
 maximum: 100
 goal: 65
 importance: 10
 , parameter: SiO2
 minimum: 2.8
 maximum: 5.8
 goal: 5.8
 importance: 1000
 , parameter: Al2O3
 minimum: 2.5
 maximum: 4.9
 goal: 4.9
 importance: 100
 , parameter: P
 minimum: 0.05
 maximum: 0.07
 goal: 0.07
 importance: 100
 , parameter: +31.5
 minimum: 8
 maximum: 10
 goal: 10
 importance: 100
 , parameter: -6.3
 minimum: 20
 maximum: 25
 goal: 25
 importance: 100
 ]
 time: 600]

In [43]:
requests = [
    out.quality for out in solution._problem.outputs
]
requests

[[parameter: Fe
  minimum: 60
  maximum: 100
  goal: 65
  importance: 10,
  parameter: SiO2
  minimum: 2.8
  maximum: 5.8
  goal: 5.8
  importance: 1000,
  parameter: Al2O3
  minimum: 2.5
  maximum: 4.9
  goal: 4.9
  importance: 100,
  parameter: P
  minimum: 0.05
  maximum: 0.07
  goal: 0.07
  importance: 100,
  parameter: +31.5
  minimum: 8
  maximum: 10
  goal: 10
  importance: 100,
  parameter: -6.3
  minimum: 20
  maximum: 25
  goal: 25
  importance: 100]]

In [46]:
max([x['start_time'] + x['duration'] for x in solution._reclaims])

91.04

In [47]:
problem.info

['Instance_006', 1000, 1]

In [ ]:
problem._time_travel